# Imports

In [1]:
import sys
import polars as pl
pl.Config.set_tbl_rows(15) # show up rows in cell
pl.Config.set_fmt_str_lengths(200)   # show up to 200 chars for string cells
import boto3
import time
from IPython.display import display
import datetime
import math
from pathlib import PurePosixPath
print(f"{sys.version=}")

sys.version='3.14.0 (main, Oct  7 2025, 09:34:52) [Clang 17.0.0 (clang-1700.3.19.1)]'


In [2]:
s3 = boto3.resource(
    's3',
    endpoint_url='http://localhost:9000',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False
)
storage_options = {
    "aws_endpoint_url": "http://localhost:9000", # Важно: aws_endpoint_url (иногда endpoint_url)
    "aws_access_key_id": "minioadmin",
    "aws_secret_access_key": "minioadmin",
    "aws_region": "us-east-1", # Для MinIO часто можно оставить us-east-1
    "aws_allow_http": "true",  # Разрешить HTTP (без SSL)
}

# Список silver S3

In [3]:
def silver_path2dt(obj):
    return datetime.datetime(**{item.split("=",1)[0]: int(item.split("=",1)[1]) for item in PurePosixPath(obj.key).parts if "=" in item})
bucket = s3.Bucket('silver')
objects = sorted(bucket.objects.all(), key=silver_path2dt)

In [4]:
objects[:2]

[s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=7/part-c3db8d99-b134-4cdf-9601-af1f29d0eaf8.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=8/part-76877218-6629-4852-88fe-28adad10a57a.parquet')]

In [5]:
total_size_mb = 0
for obj in objects:
    key = obj.key
    size_mb = obj.size//1e+6
    total_size_mb += size_mb
    print(key, size_mb, "mb,", "last_modified", obj.last_modified)

hh/vacancies/year=2025/month=12/day=7/part-c3db8d99-b134-4cdf-9601-af1f29d0eaf8.parquet 1.0 mb, last_modified 2025-12-16 10:36:01.727000+00:00
hh/vacancies/year=2025/month=12/day=8/part-76877218-6629-4852-88fe-28adad10a57a.parquet 10.0 mb, last_modified 2025-12-16 10:37:25.620000+00:00
hh/vacancies/year=2025/month=12/day=9/part-c6d99cfa-0e36-4167-a9cc-403aeb0abb38.parquet 9.0 mb, last_modified 2025-12-16 10:38:36.278000+00:00
hh/vacancies/year=2025/month=12/day=10/part-5255f869-8a86-4b1c-988b-66521df52208.parquet 9.0 mb, last_modified 2025-12-16 10:39:38.477000+00:00
hh/vacancies/year=2025/month=12/day=11/part-3fd06728-a53b-45aa-a1b5-f2f8beefcce6.parquet 8.0 mb, last_modified 2025-12-16 10:40:36.257000+00:00
hh/vacancies/year=2025/month=12/day=12/part-431ebb6e-6b9f-418a-b9de-5dd92d91127e.parquet 9.0 mb, last_modified 2025-12-16 10:41:41.757000+00:00
hh/vacancies/year=2025/month=12/day=13/part-5633c4b8-3f9c-4b88-82b1-a83e37c8d5eb.parquet 6.0 mb, last_modified 2025-12-16 10:42:41.574000+

In [6]:
print('Silver layer size', total_size_mb, 'mb')

Silver layer size 119.0 mb


# Общее кол-во вакансий в S3

In [7]:
for obj in objects:
    schema = pl.scan_parquet(f"s3://silver/{obj.key}",storage_options=storage_options).collect_schema()
    print(obj, len(schema))

s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=7/part-c3db8d99-b134-4cdf-9601-af1f29d0eaf8.parquet') 51
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=8/part-76877218-6629-4852-88fe-28adad10a57a.parquet') 51
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=9/part-c6d99cfa-0e36-4167-a9cc-403aeb0abb38.parquet') 51
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=10/part-5255f869-8a86-4b1c-988b-66521df52208.parquet') 51
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=11/part-3fd06728-a53b-45aa-a1b5-f2f8beefcce6.parquet') 51
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=12/part-431ebb6e-6b9f-418a-b9de-5dd92d91127e.parquet') 51
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=13/part-5633c4b8-3f9c-4b88-82b1-a83e37c8d5eb.parquet') 51
s3.ObjectSummary(bucket_name='silver'

In [8]:
df_lazy = pl.scan_parquet(
    [f"s3://silver/{obj.key}" for obj in objects],
    storage_options=storage_options,
)

In [9]:
df_lazy.select(
    pl.col('id').count().alias('id_cnt'),
    pl.col('id').unique().count().alias('unique_id_cnt')
).collect()

id_cnt,unique_id_cnt
u32,u32
585986,545877


In [10]:
df_lazy.tail(1).collect().glimpse()

Rows: 1
Columns: 51
$ id                                    <str> '128772562'
$ name                                  <str> 'Ведущий специалист по работе с подозрительными операциями'
$ schedule                        <struct[2]> {'id': 'remote', 'name': 'Удаленная работа'}
$ working_time_modes        <list[struct[2]]> []
$ working_time_intervals    <list[struct[2]]> []
$ working_days              <list[struct[2]]> []
$ working_hours             <list[struct[2]]> [{'id': 'HOURS_12', 'name': '12\xa0часов'}]
$ work_schedule_by_days     <list[struct[2]]> [{'id': 'TWO_ON_TWO_OFF', 'name': '2/2'}]
$ fly_in_fly_out_duration   <list[struct[2]]> []
$ is_adv_vacancy                       <bool> False
$ internship                           <bool> False
$ accept_temporary                     <bool> False
$ accept_incomplete_resumes            <bool> False
$ premium                              <bool> False
$ has_test                             <bool> False
$ show_contacts                        

# Сверка схем Bronze & Silver

In [144]:
check_report_dt = datetime.datetime(year=2025, month=12, day=15)

In [145]:
bronze_lazy = pl.scan_ndjson(f"s3://bronze/hh/vacancies/date={check_report_dt.strftime('%Y-%m-%d')}/*.jsonl.gz", storage_options=storage_options)
bronze_cols = set(bronze_lazy.collect_schema().names())

In [146]:
report_dt_obj = next(iter(s3.Bucket('silver').objects.filter(Prefix="hh/vacancies/"+check_report_dt.strftime("year=%Y/month=%m/day=%d")).all()))
silver_s3_path = report_dt_obj.key
silver_lazy = pl.scan_parquet(f"s3://silver/{silver_s3_path}", storage_options=storage_options)
silver_cols = set(silver_lazy.collect_schema().names())

In [147]:
silver_cols - bronze_cols

{'created_at_offset',
 'created_at_utc',
 'immediate_redirect_url',
 'published_at_offset',
 'published_at_utc',
 'video_vacancy'}

In [148]:
"brand_snippet" in bronze_cols

True

In [149]:
for report_dt in {"2025-12-08", "2025-12-09", "2025-12-10", "2025-12-11", "2025-12-12", "2025-12-13", "2025-12-14", "2025-12-15"}:
    print(report_dt)
    for bronze_obj in s3.Bucket('bronze').objects.filter(Prefix=f"hh/vacancies/date={report_dt}"):
        _bronze_lazy = pl.scan_ndjson(f"s3://bronze/{bronze_obj.key}/*.jsonl.gz", storage_options=storage_options)
        print("\t", "brand_snippet" in set(bronze_lazy.collect_schema().names()))

2025-12-09
	 True
	 True
	 True
	 True
	 True
	 True
2025-12-10
	 True
	 True
	 True
	 True
	 True
2025-12-13
	 True
	 True
	 True
	 True
	 True
2025-12-15
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
2025-12-11
	 True
	 True
	 True
	 True
	 True
2025-12-08
	 True
	 True
	 True
	 True
	 True
	 True
	 True
2025-12-14
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
	 True
2025-12-12
	 True
	 True
	 True
	 True
	 True


# Lil EDA

In [11]:
def silver_path2dt(obj):
    return datetime.datetime(**{item.split("=",1)[0]: int(item.split("=",1)[1]) for item in PurePosixPath(obj.key).parts if "=" in item})
bucket = s3.Bucket('silver')
objects = sorted(bucket.objects.all(), key=silver_path2dt)
objects

[s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=7/part-c3db8d99-b134-4cdf-9601-af1f29d0eaf8.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=8/part-76877218-6629-4852-88fe-28adad10a57a.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=9/part-c6d99cfa-0e36-4167-a9cc-403aeb0abb38.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=10/part-5255f869-8a86-4b1c-988b-66521df52208.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=11/part-3fd06728-a53b-45aa-a1b5-f2f8beefcce6.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=12/part-431ebb6e-6b9f-418a-b9de-5dd92d91127e.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=13/part-5633c4b8-3f9c-4b88-82b1-a83e37c8d5eb.parquet'),
 s3.ObjectSummary(bucket_name='silver', key=

In [14]:
df_lazy = (
    pl.scan_parquet([f"s3://silver/{obj.key}" for obj in objects], storage_options=storage_options)

    # Published Date transfromation from UTC to Local
    .with_columns(
        published_at_local = (
                pl.col('published_at_utc').dt.replace_time_zone(None) +
                pl.duration(minutes=pl.col('published_at_offset')
            )
        ),
        created_at_local = (
                pl.col('created_at_utc').dt.replace_time_zone(None) +
                pl.duration(minutes=pl.col('created_at_offset')
            )
        )
    )
    .with_columns(
        published_date=pl.col('published_at_local').dt.date()
    )
)

## Проверка заполненности *_offset

In [15]:
df_lazy.select("published_at_utc", "published_at_offset", "published_at_local").head(100).collect()

published_at_utc,published_at_offset,published_at_local
datetime[μs],i16,datetime[μs]
2025-12-07 09:07:46,180,2025-12-07 12:07:46
2025-12-07 01:00:49,180,2025-12-07 04:00:49
2025-12-07 04:03:28,180,2025-12-07 07:03:28
2025-12-07 08:57:08,180,2025-12-07 11:57:08
2025-12-07 06:38:09,180,2025-12-07 09:38:09
2025-12-07 10:31:57,180,2025-12-07 13:31:57
2025-12-07 05:34:02,180,2025-12-07 08:34:02
2025-12-07 12:21:22,180,2025-12-07 15:21:22
…,…,…


## Проверка дублей по id

In [16]:
df_lazy.select(
    pl.col('id').count(),
    pl.col('id').unique().count().alias('unique_id')
).collect()

id,unique_id
u32,u32
585986,545877


In [17]:
dups = df_lazy.filter(pl.col("id").is_duplicated()).sort(by='id')
dups.head(20).select("id", "name", "published_at_local", "created_at_local").collect()

id,name,published_at_local,created_at_local
str,str,datetime[μs],datetime[μs]
"""100355111""","""Продавец-кассир, пoлная занятость""",2025-12-12 08:54:39,2025-12-12 08:54:39
"""100355111""","""Продавец-кассир, пoлная занятость""",2025-12-16 15:58:36,2025-12-16 15:58:36
"""100355249""","""Продавец-кассир (ВИЗ)""",2025-12-15 13:01:47,2025-12-15 13:01:47
"""100355249""","""Продавец-кассир (ВИЗ)""",2025-12-16 14:35:14,2025-12-16 14:35:14
"""100696122""","""Оператор call-центра / Телемаркетолог/ Лидоруб""",2025-12-13 14:47:24,2025-12-13 14:47:24
"""100696122""","""Оператор call-центра / Телемаркетолог/ Лидоруб""",2025-12-16 14:47:25,2025-12-16 14:47:25
"""101064322""","""Руководитель отдела продаж""",2025-12-09 11:21:42,2025-12-09 11:21:42
"""101064322""","""Руководитель отдела продаж""",2025-12-16 15:26:34,2025-12-16 15:26:34
…,…,…,…


**Вывод**: вакансии с одинаковым id могут публиковаться в разные дни

## Describe

In [155]:
df_lazy_described = df_lazy.describe()

In [28]:
from IPython.display import display, HTML

display(HTML("""
<style>
/* classic notebook */
div.output_area pre {
  white-space: pre !important;   /* не переносить строки */
}

/* jupyterlab */
div.jp-OutputArea-output pre {
  white-space: pre !important;   /* не переносить строки */
}
</style>
"""))

with pl.Config(tbl_cols=-1, tbl_formatting='MARKDOWN', tbl_width_chars=-1):
    print(df_lazy_described)

shape: (9, 54)
| statistic  | id        | name                            | schedule | working_time_modes | working_time_intervals | working_days | working_hours | work_schedule_by_days | fly_in_fly_out_duration | is_adv_vacancy | internship | accept_temporary | accept_incomplete_resumes | premium  | has_test | show_contacts | response_letter_required | show_logo_in_search | archived | night_shifts | url                             | alternate_url                   | apply_alternate_url             | response_url                    | adv_response_url | immediate_redirect_url          | salary   | salary_range | employer | department | type     | employment | employment_form | experience | area     | address  | work_format | relations | professional_roles | contacts | adv_context | sort_point_distance | snippet  | branding | insider_interview | video_vacancy | brand_snippet | published_at_utc           | created_at_utc             | published_at_offset | created_at_offset | published_at

## Check nulls

In [156]:
df_lazy.null_count().collect().glimpse()

Rows: 1
Columns: 53
$ id                        <u32> 0
$ name                      <u32> 0
$ schedule                  <u32> 0
$ working_time_modes        <u32> 0
$ working_time_intervals    <u32> 0
$ working_days              <u32> 0
$ working_hours             <u32> 0
$ work_schedule_by_days     <u32> 0
$ fly_in_fly_out_duration   <u32> 0
$ is_adv_vacancy            <u32> 0
$ internship                <u32> 0
$ accept_temporary          <u32> 0
$ accept_incomplete_resumes <u32> 0
$ premium                   <u32> 0
$ has_test                  <u32> 0
$ show_contacts             <u32> 0
$ response_letter_required  <u32> 0
$ show_logo_in_search       <u32> 328207
$ archived                  <u32> 0
$ night_shifts              <u32> 0
$ url                       <u32> 0
$ alternate_url             <u32> 0
$ apply_alternate_url       <u32> 0
$ response_url              <u32> 509228
$ adv_response_url          <u32> 585986
$ immediate_redirect_url    <u32> 509231
$ salary                

## professional_role count

In [18]:
lens = pl.col('professional_roles').list.len()
df_lazy.select(
    lens.min().alias("min_len"),
    lens.max().alias("max_len"),
    lens.mean().alias("avg_len"),
    lens.median().alias("median_len"),
    lens.quantile(0.25).alias("p25_len"),
    lens.quantile(0.75).alias("p75_len"),
    lens.quantile(0.95).alias("p95_len"),
).collect()

min_len,max_len,avg_len,median_len,p25_len,p75_len,p95_len
u32,u32,f64,f64,f64,f64,f64
1,1,1.0,1.0,1.0,1.0,1.0


In [19]:
with pl.Config(set_tbl_rows=15):
    res = df_lazy.select("professional_roles").head(15).collect()
    display(res)

professional_roles
list[struct[2]]
"[{""115"",""Слесарь, сантехник""}]"
"[{""130"",""Уборщица, уборщик""}]"
"[{""21"",""Водитель""}]"
"[{""97"",""Продавец-консультант, продавец-кассир""}]"
"[{""21"",""Водитель""}]"
"[{""17"",""Бизнес-тренер""}]"
"[{""85"",""Оператор производственной линии""}]"
"[{""70"",""Менеджер по продажам, менеджер по работе с клиентами""}]"
"[{""70"",""Менеджер по продажам, менеджер по работе с клиентами""}]"


In [20]:
q = (df_lazy.with_columns(
        pl.col('professional_roles').list.get(0).struct.field('name').alias('professional_role')
    )
)
with pl.Config(set_tbl_rows=20):
    res = q.group_by('professional_role').len().sort(by="len", descending=True).collect()
    display(res)

professional_role,len
str,u32
"""Продавец-консультант, продавец-кассир""",50041
"""Менеджер по продажам, менеджер по работе с клиентами""",37387
"""Другое""",36760
"""Курьер""",28404
"""Водитель""",27529
"""Оператор производственной линии""",27183
"""Упаковщик, комплектовщик""",25289
"""Повар, пекарь, кондитер""",23360
"""Разнорабочий""",19988


## Schema

In [22]:
df = pl.concat([df_lazy.head(1_000), df_lazy.tail(1_000)]).collect()
df.shape

(2000, 54)

In [23]:
print(f"{df.estimated_size("mb"):.1f} MB")

14.8 MB


In [24]:
df.sample(1).glimpse()

Rows: 1
Columns: 54
$ id                                    <str> '128637424'
$ name                                  <str> 'Электромонтажник в Новую Чару'
$ schedule                        <struct[2]> {'id': 'flyInFlyOut', 'name': 'Вахтовый метод'}
$ working_time_modes        <list[struct[2]]> []
$ working_time_intervals    <list[struct[2]]> []
$ working_days              <list[struct[2]]> []
$ working_hours             <list[struct[2]]> [{'id': 'HOURS_11', 'name': '11 часов'}]
$ work_schedule_by_days     <list[struct[2]]> [{'id': 'OTHER', 'name': 'Другое'}]
$ fly_in_fly_out_duration   <list[struct[2]]> [{'id': 'DAYS_60', 'name': '60'}]
$ is_adv_vacancy                       <bool> False
$ internship                           <bool> False
$ accept_temporary                     <bool> False
$ accept_incomplete_resumes            <bool> True
$ premium                              <bool> False
$ has_test                             <bool> False
$ show_contacts                        <boo

In [99]:
df.sample(1)

id,name,schedule,working_time_modes,working_time_intervals,working_days,working_hours,work_schedule_by_days,fly_in_fly_out_duration,is_adv_vacancy,internship,accept_temporary,accept_incomplete_resumes,premium,has_test,show_contacts,response_letter_required,show_logo_in_search,archived,night_shifts,url,alternate_url,apply_alternate_url,response_url,adv_response_url,immediate_redirect_url,salary,salary_range,employer,department,type,employment,employment_form,experience,area,address,work_format,relations,professional_roles,contacts,adv_context,sort_point_distance,snippet,branding,insider_interview,video_vacancy,brand_snippet,published_at_utc,created_at_utc,published_at_offset,created_at_offset,published_at_local,created_at_local,published_date
str,str,struct[2],list[struct[2]],list[struct[2]],list[struct[2]],list[struct[2]],list[struct[2]],list[struct[2]],bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,str,str,str,str,str,str,struct[4],struct[6],struct[9],struct[2],struct[2],struct[2],struct[2],struct[2],struct[3],struct[10],list[struct[2]],list[str],list[struct[2]],struct[3],str,f64,struct[2],struct[2],struct[2],struct[7],struct[7],datetime[μs],datetime[μs],i16,i16,datetime[μs],datetime[μs],date
"""128636262""","""Бетонщик (Губкинский)""","{""flyInFlyOut"",""Вахтовый метод""}",[],[],[],"[{""HOURS_11"",""11 часов""}]","[{""OTHER"",""Другое""}]","[{""DAYS_60"",""60""}]",false,false,false,true,false,false,true,false,null,false,false,"""https://api.hh.ru/vacancies/128636262?host=hh.ru""","""https://hh.ru/vacancy/128636262""","""https://hh.ru/applicant/vacancy_response?vacancyId=128636262""",null,null,null,"{198000,null,""RUR"",false}","{198000,null,""RUR"",false,{""MONTH"",""За месяц""},{""TWICE_PER_MONTH"",""Два раза в месяц""}}","{""11875449"",""ПрессБук"",""https://api.hh.ru/employers/11875449"",""https://hh.ru/employer/11875449"",{""https://img.hhcdn.ru/employer-logo-original/1402503.png"",""https://img.hhcdn.ru/employer-logo/7229684.png"",""https://img.hhcdn.ru/employer-logo/7229685.png""},""https://api.hh.ru/vacancies?employer_id=11875449"",1,false,true}",null,"{""open"",""Открытая""}","{""full"",""Полная занятость""}","{""FLY_IN_FLY_OUT"",""Вахта""}","{""between1And3"",""От 1 года до 3 лет""}","{""1290"",""Нижние Серги"",""https://api.hh.ru/areas/1290""}","{""Губкинский"",null,null,64.432625,76.500503,null,""Губкинский"",null,[],""21085174""}",[],[],"[{""102"",""Разнорабочий""}]","{null,null,[]}",null,null,"{null,null}",null,null,null,null,2025-12-16 14:08:30,2025-12-16 14:08:30,180,180,2025-12-16 17:08:30,2025-12-16 17:08:30,2025-12-16


# Try DuckDB EDA

In [2]:
import duckdb
print(f"{duckdb.__version__=}")
con = duckdb.connect()
con.execute("""
    INSTALL httpfs;
    LOAD httpfs;
    
    CREATE OR REPLACE SECRET minio (
      TYPE s3,
      PROVIDER config,
      KEY_ID 'minioadmin',
      SECRET 'minioadmin',
      REGION 'us-east-1',
      ENDPOINT 'localhost:9000',
      USE_SSL false,
      URL_STYLE 'path'
    );
""")

duckdb.__version__='1.4.3'


In [40]:
silver_parquets = [f"s3://silver/{obj.key}" for obj in objects]

In [43]:
silver_parquets[0]

's3://silver/hh/vacancies/year=2025/month=12/day=7/part-c3db8d99-b134-4cdf-9601-af1f29d0eaf8.parquet'

In [3]:
con.sql("select * from read_parquet('s3://silver/hh/vacancies/year=2025/month=12/day=7/part-c3db8d99-b134-4cdf-9601-af1f29d0eaf8.parquet') limit 1")

┌───────────┬─────────────────────────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬────────────────────────────────────────┬──────────────────────────────────────┬────────────────┬────────────┬──────────────────┬───────────────────────────┬─────────┬──────────┬───────────────┬──────────────────────────┬─────────────────────┬──────────┬──────────────┬──────────────────────────────────────────────────┬─────────────────────────────────┬──────────────────────────────────────────────────────────────┬──────────────┬──────────────────┬────────────────────────┬─────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────────

## Кол-во вакансий за 2025-12

In [47]:
con.sql("select count(*) rows_cnt, count(distinct id) unique_id_cnt from read_parquet('s3://silver/hh/vacancies/year=2025/month=12/day=*/*.parquet')")

┌──────────┬───────────────┐
│ rows_cnt │ unique_id_cnt │
│  int64   │     int64     │
├──────────┼───────────────┤
│   585986 │        545877 │
└──────────┴───────────────┘

In [89]:
con.sql("describe select * from read_parquet('s3://silver/hh/vacancies/year=2025/month=12/day=*/*.parquet')").show(max_rows=100)

┌───────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│        column_name        │                                                                                                                                                                                                                                     column_type                                                                                                                                                                                               

In [4]:
vacancies = con.sql("select * from read_parquet('s3://silver/hh/vacancies/year=2025/month=12/day=*/*.parquet')")

## Кол-во null значений

In [115]:
null_cnt_sql = ",\n".join([
    f"count(*) filter (where {col} is null) as {col}"
    for col in vacancies.columns
])
null_cnt_pd = con.sql(f"select {null_cnt_sql} from vacancies").to_df().T
null_cnt_pd["total_cnt"] = con.sql("select count(*) cnt from vacancies").fetchone()[0]
null_cnt_pd.columns = ["null_cnt", "total_cnt"]
null_cnt_pd["null_pcnt"] = round(null_cnt_pd.null_cnt/null_cnt_pd.total_cnt*100, 1)
null_cnt_pd

,null_cnt,total_cnt,null_pcnt
id,0,585986,0.0
name,0,585986,0.0
schedule,0,585986,0.0
working_time_modes,0,585986,0.0
working_time_intervals,0,585986,0.0
working_days,0,585986,0.0
working_hours,0,585986,0.0
work_schedule_by_days,0,585986,0.0
fly_in_fly_out_duration,0,585986,0.0
is_adv_vacancy,0,585986,0.0


## Кол-во пустых STRUCT[] колонок

**Вывод:** в основном заполняются колонки working_hours, work_schedule_by_days, work_format

In [135]:
describe_pd = con.sql("describe select * from read_parquet('s3://silver/hh/vacancies/year=2025/month=12/day=*/*.parquet')").to_df()

In [124]:
column_type_is_list_mask = describe_pd.column_type.apply(lambda x: x.endswith("[]"))
lists_columns = describe_pd[column_type_is_list_mask].column_name.tolist()
describe_pd[column_type_is_list_mask]

,column_name,column_type,null,key,default,extra
3,working_time_modes,"STRUCT(id VARCHAR, ""name"" VARCHAR)[]",YES,None,None,None
4,working_time_intervals,"STRUCT(id VARCHAR, ""name"" VARCHAR)[]",YES,None,None,None
5,working_days,"STRUCT(id VARCHAR, ""name"" VARCHAR)[]",YES,None,None,None
6,working_hours,"STRUCT(id VARCHAR, ""name"" VARCHAR)[]",YES,None,None,None
7,work_schedule_by_days,"STRUCT(id VARCHAR, ""name"" VARCHAR)[]",YES,None,None,None
8,fly_in_fly_out_duration,"STRUCT(id VARCHAR, ""name"" VARCHAR)[]",YES,None,None,None
36,work_format,"STRUCT(id VARCHAR, ""name"" VARCHAR)[]",YES,None,None,None
37,relations,VARCHAR[],YES,None,None,None
38,professional_roles,"STRUCT(id VARCHAR, ""name"" VARCHAR)[]",YES,None,None,None


In [134]:
empty_lists_sql = ",\n".join([
    f"count(*) filter (where length({col})=0) as {col}"
    for col in lists_columns
])
empty_lists_pd = con.sql(f"select {empty_lists_sql} from vacancies").to_df().T
empty_lists_pd["total_cnt"] = con.sql("select count(*) cnt from vacancies").fetchone()[0]
empty_lists_pd.columns = ["empty_list_cnt", "total_cnt"]
empty_lists_pd["empty_pcnt"] = round(empty_lists_pd.empty_list_cnt/null_cnt_pd.total_cnt*100, 1)
empty_lists_pd

,empty_list_cnt,total_cnt,empty_pcnt
working_time_modes,566233,585986,96.6
working_time_intervals,528809,585986,90.2
working_days,585085,585986,99.8
working_hours,8542,585986,1.5
work_schedule_by_days,7862,585986,1.3
fly_in_fly_out_duration,508132,585986,86.7
work_format,119991,585986,20.5
relations,585986,585986,100.0
professional_roles,0,585986,0.0


## Null & empty struct

In [157]:
null_and_empty_struct = null_cnt_pd.join(empty_lists_pd[["empty_list_cnt", "empty_pcnt"]], rsuffix="_empty")
null_and_empty_struct = null_and_empty_struct[['total_cnt', 'null_cnt', 'null_pcnt', 'empty_list_cnt', 'empty_pcnt']]
null_and_empty_struct

,total_cnt,null_cnt,null_pcnt,empty_list_cnt,empty_pcnt
id,585986,0,0.0,NaN,NaN
name,585986,0,0.0,NaN,NaN
schedule,585986,0,0.0,NaN,NaN
working_time_modes,585986,0,0.0,566233.0,96.6
working_time_intervals,585986,0,0.0,528809.0,90.2
working_days,585986,0,0.0,585085.0,99.8
working_hours,585986,0,0.0,8542.0,1.5
work_schedule_by_days,585986,0,0.0,7862.0,1.3
fly_in_fly_out_duration,585986,0,0.0,508132.0,86.7
is_adv_vacancy,585986,0,0.0,NaN,NaN


## Формат работы / schedule

In [158]:
con.sql("select schedule, count(*) rows_cnt, count(distinct id) unique_id_cnt from vacancies group by all order by rows_cnt desc")

┌─────────────────────────────────────────────┬──────────┬───────────────┐
│                  schedule                   │ rows_cnt │ unique_id_cnt │
│     struct(id varchar, "name" varchar)      │  int64   │     int64     │
├─────────────────────────────────────────────┼──────────┼───────────────┤
│ {'id': fullDay, 'name': Полный день}        │   334374 │        309230 │
│ {'id': flyInFlyOut, 'name': Вахтовый метод} │    77854 │         73068 │
│ {'id': shift, 'name': Сменный график}       │    73120 │         69572 │
│ {'id': flexible, 'name': Гибкий график}     │    69974 │         65255 │
│ {'id': remote, 'name': Удаленная работа}    │    30664 │         28770 │
└─────────────────────────────────────────────┴──────────┴───────────────┘

## List structs

In [180]:
def list_struct_report(col):
    return con.sql(f"select {col}, length({col}) len, count(*) cnt from vacancies group by all order by cnt desc limit 10")

### working_time_modes

In [181]:
list_struct_report("working_time_modes")

┌────────────────────────────────────────────────────────────────────┬───────┬────────┐
│                         working_time_modes                         │  len  │  cnt   │
│                struct(id varchar, "name" varchar)[]                │ int64 │ int64  │
├────────────────────────────────────────────────────────────────────┼───────┼────────┤
│ []                                                                 │     0 │ 566233 │
│ [{'id': start_after_sixteen, 'name': 'С началом дня после 16:00'}] │     1 │  19753 │
└────────────────────────────────────────────────────────────────────┴───────┴────────┘

### working_time_intervals

In [182]:
list_struct_report("working_time_intervals")

┌──────────────────────────────────────────────────────────────────────────────────────┬───────┬────────┐
│                                working_time_intervals                                │  len  │  cnt   │
│                         struct(id varchar, "name" varchar)[]                         │ int64 │ int64  │
├──────────────────────────────────────────────────────────────────────────────────────┼───────┼────────┤
│ []                                                                                   │     0 │ 528809 │
│ [{'id': from_four_to_six_hours_in_a_day, 'name': Можно сменами по 4-6 часов в день}] │     1 │  57177 │
└──────────────────────────────────────────────────────────────────────────────────────┴───────┴────────┘

### working_days

In [183]:
list_struct_report("working_days")

┌────────────────────────────────────────────────────────────────────────┬───────┬────────┐
│                              working_days                              │  len  │  cnt   │
│                  struct(id varchar, "name" varchar)[]                  │ int64 │ int64  │
├────────────────────────────────────────────────────────────────────────┼───────┼────────┤
│ []                                                                     │     0 │ 585085 │
│ [{'id': only_saturday_and_sunday, 'name': По субботам и воскресеньям}] │     1 │    901 │
└────────────────────────────────────────────────────────────────────────┴───────┴────────┘

### working_hours

In [184]:
list_struct_report("working_hours")

┌───────────────────────────────────────────────────────────────────────────────────────────────────────┬───────┬────────┐
│                                             working_hours                                             │  len  │  cnt   │
│                                 struct(id varchar, "name" varchar)[]                                  │ int64 │ int64  │
├───────────────────────────────────────────────────────────────────────────────────────────────────────┼───────┼────────┤
│ [{'id': HOURS_8, 'name': 8 часов}]                                                                    │     1 │ 244569 │
│ [{'id': HOURS_12, 'name': 12 часов}]                                                                  │     1 │ 101800 │
│ [{'id': HOURS_11, 'name': 11 часов}]                                                                  │     1 │  42404 │
│ [{'id': OTHER, 'name': Другое}]                                                                       │     1 │  25201 │
│ [{'id': HOURS_

### work_schedule_by_days

In [185]:
list_struct_report("work_schedule_by_days")

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────┬────────┐
│                                                                                                                                                                                                                                                              work_schedule_by_days                                                                                                                                                                              

### fly_in_fly_out_duration

In [187]:
list_struct_report("fly_in_fly_out_duration")

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────┬────────┐
│                                                                 fly_in_fly_out_duration                                                                 │  len  │  cnt   │
│                                                          struct(id varchar, "name" varchar)[]                                                           │ int64 │ int64  │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼───────┼────────┤
│ []                                                                                                                                                      │     0 │ 508132 │
│ [{'id': DAYS_30, 'name': 30}]                                                                                                        

## is_adv_vacancy

In [189]:
con.sql("select is_adv_vacancy, count(*) cnt from vacancies group by all order by cnt desc")

┌────────────────┬──────────────┐
│ is_adv_vacancy │ count_star() │
│    boolean     │    int64     │
├────────────────┼──────────────┤
│ false          │       585986 │
└────────────────┴──────────────┘

## internship

In [191]:
con.sql("select internship, count(*) cnt from vacancies group by all order by cnt desc")

┌────────────┬────────┐
│ internship │  cnt   │
│  boolean   │ int64  │
├────────────┼────────┤
│ false      │ 562110 │
│ true       │  23876 │
└────────────┴────────┘

## accept_temporary

In [192]:
con.sql("select accept_temporary, count(*) cnt from vacancies group by all order by cnt desc")

┌──────────────────┬────────┐
│ accept_temporary │  cnt   │
│     boolean      │ int64  │
├──────────────────┼────────┤
│ false            │ 525486 │
│ true             │  60500 │
└──────────────────┴────────┘

## Описание данных

Ниже представлена сопоставленная таблица. За основу взята **Схема данных**, так как она определяет физическую структуру таблицы. Описания подтянуты из списка «Описание колонок».

Для полей, которые есть в схеме, но отсутствуют в текстовом описании, проставлена пометка «Нет в описании».

| Колонка | Тип данных (из схемы) | Описание | Примечание |
| :--- | :--- | :--- | :--- |
| **id** | `VARCHAR` | Идентификатор вакансии | Required |
| **name** | `VARCHAR` | Название | Required |
| **schedule** | `STRUCT(id VARCHAR, "name" VARCHAR)` | График работы | **Deprecated**. Объект или null |
| **working_time_modes** | `STRUCT(id VARCHAR, "name" VARCHAR)[]` | Список режимов времени работы | **Deprecated**. Массив объектов или null |
| **working_time_intervals** | `STRUCT(id VARCHAR, "name" VARCHAR)[]` | Список с временными интервалами работы | **Deprecated**. Массив объектов или null |
| **working_days** | `STRUCT(id VARCHAR, "name" VARCHAR)[]` | Список рабочих дней | **Deprecated**. Массив объектов или null |
| **working_hours** | `STRUCT(id VARCHAR, "name" VARCHAR)[]` | Список вариантов рабочих часов в день | Массив объектов или null |
| **work_schedule_by_days** | `STRUCT(id VARCHAR, "name" VARCHAR)[]` | Список графиков работы | Массив объектов или null |
| **fly_in_fly_out_duration** | `STRUCT(id VARCHAR, "name" VARCHAR)[]` | Варианты длительности вахты | Массив объектов или null |
| **is_adv_vacancy** | `BOOLEAN` | — | Нет в описании |
| **internship** | `BOOLEAN` | Стажировка | |
| **accept_temporary** | `BOOLEAN` | Временное трудоустройство | Указание, что вакансия доступна с временным трудоустройством |
| **accept_incomplete_resumes** | `BOOLEAN` | Неполное резюме | Разрешен ли отклик на вакансию неполным резюме. Required |
| **premium** | `BOOLEAN` | Премиум-вакансия | Является ли данная вакансия премиум-вакансией |
| **has_test** | `BOOLEAN` | Наличие теста | Информация о наличии прикрепленного тестового задания. Required |
| **show_contacts** | `BOOLEAN` | Доступны ли контакты | |
| **response_letter_required** | `BOOLEAN` | Сопроводительное письмо | Обязательно ли заполнять сообщение при отклике. Required |
| **show_logo_in_search** | `BOOLEAN` | Отображать ли лого | Отображать ли лого для вакансии в поисковой выдаче |
| **archived** | `BOOLEAN` | В архиве | Находится ли данная вакансия в архиве |
| **night_shifts** | `BOOLEAN` | Ночные смены | |
| **url** | `VARCHAR` | URL вакансии | Required |
| **alternate_url** | `VARCHAR` | Ссылка на представление вакансии на сайте | Required |
| **apply_alternate_url** | `VARCHAR` | Ссылка на отклик на вакансию на сайте | Required |
| **response_url** | `VARCHAR` | URL отклика | Для прямых вакансий (type.id=direct) |
| **adv_response_url** | `VARCHAR` | — | Нет в описании |
| **immediate_redirect_url** | `VARCHAR` | — | Нет в описании |
| **salary** | `STRUCT("from" BIGINT, "to" BIGINT, currency VARCHAR, gross BOOLEAN)` | Зарплата | **Deprecated**. Required |
| **salary_range** | `STRUCT(...)` | Зарплата | Required. Расширенная структура с mode и frequency |
| **employer** | `STRUCT(...)` | Информация о компании работодателя | Required. Содержит ссылки на логотипы, аккредитацию и др. |
| **department** | `STRUCT(id VARCHAR, "name" VARCHAR)` | Департамент | Required. Может быть null-объектом |
| **type** | `STRUCT(id VARCHAR, "name" VARCHAR)` | Тип вакансии | Required |
| **employment** | `STRUCT(id VARCHAR, "name" VARCHAR)` | Тип занятости | **Deprecated** |
| **employment_form** | `STRUCT(id VARCHAR, "name" VARCHAR)` | Тип занятости | |
| **experience** | `STRUCT(id VARCHAR, "name" VARCHAR)` | Опыт работы | |
| **area** | `STRUCT(id VARCHAR, "name" VARCHAR, url VARCHAR)` | Регион | Required |
| **address** | `STRUCT(...)` | Адрес | В схеме включает в себя массив `metro_stations` (станции метро) |
| **work_format** | `STRUCT(id VARCHAR, "name" VARCHAR)[]` | Список форматов работы | Массив объектов |
| **relations** | `VARCHAR[]` | Связи соискателя с вакансией | Required. Enum: favorited, got_response и др. |
| **professional_roles** | `STRUCT(id VARCHAR, "name" VARCHAR)[]` | Список профессиональных ролей | Required. Массив объектов |
| **contacts** | `STRUCT(...)` | Контактная информация | Включает телефоны, email. Объект или null |
| **adv_context** | `VARCHAR` | — | Нет в описании |
| **sort_point_distance** | `DOUBLE` | Расстояние (в метрах) | Между центром сортировки и адресом вакансии |
| **snippet** | `STRUCT(requirement VARCHAR, responsibility VARCHAR)` | Дополнительные текстовые отрывки | Required. Требования и обязанности |
| **branding** | `STRUCT("type" VARCHAR, tariff VARCHAR)` | — | Нет в описании |
| **insider_interview** | `STRUCT(id VARCHAR, url VARCHAR)` | Интервью о жизни в компании | Объект или null |
| **video_vacancy** | `STRUCT(...)` | Видео вакансия | Содержит ссылки на видео, обложки и превью |
| **brand_snippet** | `STRUCT(...)` | — | Нет в описании (брендинговые элементы: лого, фон) |
| **published_at_utc** | `TIMESTAMP` | Дата и время публикации | В описании поле названо `published_at` (Required) |
| **created_at_utc** | `TIMESTAMP` | Дата и время создания | В описании поле названо `created_at` |
| **published_at_offset** | `SMALLINT` | — | Техническое поле (смещение часового пояса), нет в описании |
| **created_at_offset** | `SMALLINT` | — | Техническое поле (смещение часового пояса), нет в описании |
| **day** | `BIGINT` | — | Техническое поле партицирования, нет в описании |
| **month** | `BIGINT` | — | Техническое поле партицирования, нет в описании |
| **year** | `BIGINT` | — | Техническое поле партицирования, нет в описании |

## Deprecated columns by hh.ru API Docs

In [193]:
deprecated_columns = [
    "employment",
    "salary",
    "schedule",
    "working_days",
    "working_time_intervals",
    "working_time_modes",
]

In [196]:
deprecated_columns_sql = ",".join(deprecated_columns)
con.sql(f"select {deprecated_columns_sql} from vacancies order by random() limit 10")

┌───────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────┐
│                employment                 │                               salary                                │                  schedule                   │             working_days             │                                working_time_intervals                                │          working_time_modes          │
│    struct(id varchar, "name" varchar)     │ struct("from" bigint, "to" bigint, currency varchar, gross boolean) │     struct(id varchar, "name" varchar)      │ struct(id varchar, "name" varchar)[] │                         struct(id varchar, "name" varchar)[]                         │ struct(id varchar, "name" varchar)[] │
├──────────────────

## snippet

In [208]:
con.sql("""
    select
        count(snippet.requirement) rows_cnt,
        count(distinct snippet.requirement) unique_cnt,
        count(*) filter (where snippet.requirement is null) null_cnt,
        round(null_cnt/rows_cnt*100, 2) null_prcnt
    from vacancies
""")

┌──────────┬────────────┬──────────┬────────────┐
│ rows_cnt │ unique_cnt │ null_cnt │ null_prcnt │
│  int64   │   int64    │  int64   │   double   │
├──────────┼────────────┼──────────┼────────────┤
│   526072 │     262576 │    59914 │      11.39 │
└──────────┴────────────┴──────────┴────────────┘

In [210]:
con.sql("select snippet.requirement from vacancies order by random() limit 10")

┌───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                                                            requirement                                                                                            │
│                                                                                              varchar                                                                                              │
├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ УМЕНИЕ РАБОТАТЬ В УСЛОВИЯХ МНОГОЗАДАЧНОСТИ. УМЕНИЕ и ЖЕЛАНИЕ создавать и поддерживать АТМОСФЕРУ ГОСТЕПРИИМСТВА. Активный и конструктивный подход к решению служебных...                           │
│ Высшее т

In [211]:
con.sql("""
    select
        count(snippet.responsibility) rows_cnt,
        count(distinct snippet.responsibility) unique_cnt,
        count(*) filter (where snippet.responsibility is null) null_cnt,
        round(null_cnt/rows_cnt*100, 2) null_prcnt
    from vacancies
""")

┌──────────┬────────────┬──────────┬────────────┐
│ rows_cnt │ unique_cnt │ null_cnt │ null_prcnt │
│  int64   │   int64    │  int64   │   double   │
├──────────┼────────────┼──────────┼────────────┤
│   559999 │     273205 │    25987 │       4.64 │
└──────────┴────────────┴──────────┴────────────┘

In [213]:
con.sql("select snippet.responsibility from vacancies order by random() limit 10")

┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                                                               responsibility                                                                                               │
│                                                                                                  varchar                                                                                                   │
├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ Работы в соответствии с должностной инструкцией каменщика в строительстве.                                                                                                

## professional_roles

In [228]:
con.sql("select max(length(professional_roles)) max_roles_cnt from vacancies ")

┌───────────────┐
│ max_roles_cnt │
│     int64     │
├───────────────┤
│             1 │
└───────────────┘

In [6]:
con.sql("select professional_roles from vacancies order by random() limit 10")

┌───────────────────────────────────────────────────────────────┐
│                      professional_roles                       │
│             struct(id varchar, "name" varchar)[]              │
├───────────────────────────────────────────────────────────────┤
│ [{'id': 142, 'name': Экономист}]                              │
│ [{'id': 97, 'name': 'Продавец-консультант, продавец-кассир'}] │
│ [{'id': 97, 'name': 'Продавец-консультант, продавец-кассир'}] │
│ [{'id': 51, 'name': Кассир-операционист}]                     │
│ [{'id': 131, 'name': 'Упаковщик, комплектовщик'}]             │
│ [{'id': 52, 'name': Кладовщик}]                               │
│ [{'id': 84, 'name': 'Оператор ПК, оператор базы данных'}]     │
│ [{'id': 97, 'name': 'Продавец-консультант, продавец-кассир'}] │
│ [{'id': 96, 'name': 'Программист, разработчик'}]              │
│ [{'id': 161, 'name': Руководитель филиала}]                   │
├───────────────────────────────────────────────────────────────┤
│         

In [242]:
con.sql("""
    select
        count(professional_roles) rows_cnt,
        count(distinct professional_roles) unique_cnt,
        count(distinct professional_roles[1].name) unique_names_cnt,
        count(distinct professional_roles[1].id) unique_ids_cnt
    from vacancies
""")

┌──────────┬────────────┬──────────────────┬────────────────┐
│ rows_cnt │ unique_cnt │ unique_names_cnt │ unique_ids_cnt │
│  int64   │   int64    │      int64       │     int64      │
├──────────┼────────────┼──────────────────┼────────────────┤
│   585986 │        174 │              174 │            174 │
└──────────┴────────────┴──────────────────┴────────────────┘

In [245]:
con.sql("select professional_roles[1].name as name, count(*) cnt from vacancies group by all order by cnt desc")

┌──────────────────────────────────────────────────────┬───────┐
│                         name                         │  cnt  │
│                       varchar                        │ int64 │
├──────────────────────────────────────────────────────┼───────┤
│ Продавец-консультант, продавец-кассир                │ 50041 │
│ Менеджер по продажам, менеджер по работе с клиентами │ 37387 │
│ Другое                                               │ 36760 │
│ Курьер                                               │ 28404 │
│ Водитель                                             │ 27529 │
│ Оператор производственной линии                      │ 27183 │
│ Упаковщик, комплектовщик                             │ 25289 │
│ Повар, пекарь, кондитер                              │ 23360 │
│ Разнорабочий                                         │ 19988 │
│ Бухгалтер                                            │ 11525 │
│     ·                                                │    ·  │
│     ·                  

In [251]:
con.sql("select name from vacancies where professional_roles[1].name = 'Другое' order by random() limit 10")

┌─────────────────────────────────────────────────────────────────────────────────┐
│                                      name                                       │
│                                     varchar                                     │
├─────────────────────────────────────────────────────────────────────────────────┤
│ Актер / актриса на YouTube канал (Actress/Actor)                                │
│ Заведующий складом (ГСМ)                                                        │
│ Слесарь-ремонтник участка ТО и ремонта оборудования                             │
│ Менеджер пункта выдачи заказов Ozon                                             │
│ Специалист контакт-центра Организатор перевозок                                 │
│ Мастер по благоустройству                                                       │
│ Руководитель отдела входного контроля                                           │
│ Младший специалист в области управления инвестиционными проектами/строител

In [250]:
con.sql("select name from vacancies where professional_roles[1].name = 'Продавец-консультант, продавец-кассир' order by random() limit 10")

┌──────────────────────────────────────────────────────────────────────┐
│                                 name                                 │
│                               varchar                                │
├──────────────────────────────────────────────────────────────────────┤
│ Продавец-консультант отдела МясновЪ (м. Орехово)                     │
│ Продавец - консультант отдела Мясновъ (ул. Максима Горького, 218/22) │
│ Старший продавец (Москва, Краснодонская, 16/19)                      │
│ Продавец (Гоголя, 2В)                                                │
│ Продавец - кассир (пр-т Кузнецова)                                   │
│ Старший продавец-кассир                                              │
│ Продавец (Дубна, Боголюбова, 19А)                                    │
│ Продавец-консультант (м. Войковская, Новопетровская, 3)              │
│ Сотрудник торгового зала                                             │
│ Продавец - кассир (Электроугли)                  

In [248]:
con.sql("select name from vacancies where professional_roles[1].name = 'Курьер' order by random() limit 10")

┌─────────────────────────────────────────────────────┐
│                        name                         │
│                       varchar                       │
├─────────────────────────────────────────────────────┤
│ Водитель-курьер на личном автомобиле (Бум)          │
│ Курьер посылок                                      │
│ Водитель-курьер на личном автомобиле                │
│ Курьер в доставку документов и канцелярии (посылки) │
│ Водитель-курьер на личном автомобиле                │
│ Автокурьер                                          │
│ Курьер Маркет Деливери                              │
│ Курьер на автомобиле компании (Волгоградский)       │
│ Экспресс-курьер к партнеру Яндекс.Еда               │
│ Курьер к партнеру Яндекс.Еда                        │
├─────────────────────────────────────────────────────┤
│                       10 rows                       │
└─────────────────────────────────────────────────────┘